<a href="https://colab.research.google.com/github/ac1094/Vid-Processing/blob/main/Inconsistency_Detector1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary Inconsistency Detector DNN

In [168]:
import import_ipynb
from Functions import *
from ConsistencyIndexes import *

## Resize Image Shape

In [169]:
h,w = 100,200

## CNN for 9 channel input and single value output

In [170]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as Fun
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN Model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=9, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 12 * 25, 256)  # Assuming input size (9, 100, 200) and 3 pooling layers
        self.fc2 = nn.Linear(256, 1)
        
    def forward(self, x):
        x = self.pool(Fun.sigmoid(self.conv1(x)))
        x = self.pool(Fun.sigmoid(self.conv2(x)))
        x = self.pool(Fun.sigmoid(self.conv3(x)))
        x = x.view(-1, 128 * 12 * 25)  # Flattening
        x = Fun.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Define the Training Function
def train_cnn(model, train_inputs, train_targets, epochs=10, batch_size=16, lr=0.001):
    # Convert inputs and targets to tensors
    inputs = torch.stack(train_inputs)
    targets = torch.tensor(train_targets).float().view(-1, 1)

    # Create DataLoader
    dataset = TensorDataset(inputs, targets)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Define loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Training loop
    model.train()
    for epoch in range(epochs):
        for i, (x_batch, y_batch) in enumerate(loader):
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

    print("Training complete")

# Define the Prediction Function
def predict_cnn(model, test_inputs):
    model.eval()
    inputs = torch.stack(test_inputs)
    with torch.no_grad():
        outputs = model(inputs)
    predictions = (outputs > 0.5).int()  # Convert to binary output
    return predictions.view(-1).tolist()

## Read the images, Add Inconsistencies and create moved frames

In [171]:
Frames = read_images("saved_frames")
F,_ = AddInc(Frames,3)
F = [cv2.resize(f,(w,h),interpolation=cv2.INTER_AREA) for f in F]
F0 = np.asarray(F[:-2],dtype=np.uint8)
F2 = np.asarray(F[2:],dtype=np.uint8)
F = np.asarray(F[1:-1],dtype=np.uint8)

## Create input of past, present and future frames

In [172]:
Input = [np.asarray((F0[i][:,:,0],F0[i][:,:,1],F0[i][:,:,2],
                    F[i][:,:,0],F[i][:,:,1],F[i][:,:,2],
                    F2[i][:,:,0],F2[i][:,:,1],F2[i][:,:,2],
                     )) for i in range(len(F))]

In [173]:
print(Input[0].shape)
print(len(Input))

(9, 100, 200)
67


## Create list of torch arrays as input

In [174]:
input_arrays = [torch.tensor(i,dtype=torch.float32) for i in Input[:40]]
print(len(input_arrays),input_arrays[0].shape)

40 torch.Size([9, 100, 200])


## Create list of target values

In [175]:
target_arrays = [1 if i%3==0 and i!=0 else 0 for i in range(len(Input[:40]))]
print(target_arrays)

[0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]


## Instance of model

In [176]:
model = SimpleCNN()

## Train Model

In [177]:
train_cnn(model, input_arrays, target_arrays, epochs=10, batch_size=16, lr=0.001)

Epoch 1/10, Loss: 1.124044
Epoch 2/10, Loss: 0.529941
Epoch 3/10, Loss: 1.904245
Epoch 4/10, Loss: 1.732984
Epoch 5/10, Loss: 0.803431
Epoch 6/10, Loss: 0.399431
Epoch 7/10, Loss: 0.669811
Epoch 8/10, Loss: 1.110191
Epoch 9/10, Loss: 0.383389
Epoch 10/10, Loss: 0.566384
Training complete


## Predict Output

In [178]:
predicted_output = predict_cnn(model,input_arrays)
print(predicted_output)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
